# 네이버 기사 분석2 - 기사 텍스트 정제
## gensim을 이용한 토픽모델링 분석
: 토픽 모델링을 적용하기 위해 텍스트를 처리해야 하는데 라이브러리인 gensim을 사용해본다. 

### 1. 토픽 모델링을 위한 라이브러리 불러오기

In [15]:
pip install gensim #gensim 설치

Note: you may need to restart the kernel to use updated packages.


In [17]:
pip install tqdm #파이썬에 있는 소스중 하나인 tqdm을 이용하여 남은 시간과 진행정도를 한눈에 확인


Note: you may need to restart the kernel to use updated packages.


In [18]:
from tqdm import tqdm_notebook #process bar
from konlpy.tag import Mecab #Mecab 등 형태소 분석기 불러오기
import string #특수문자를 클리닝에서 제거해주고 싶을 때 사용하려고 특수문자를 불러온다. 
import warnings #warning 제거

from gensim import corpora #gensim에서 사용하는 vectorizer 모듈과, LDA model을 불러온다. #corpora : data를 가져와서 gensim에서 LDA를 돌릴 수 있게 만들어주는 것
from gensim import models #models : 여기에 LDA가 있다. 

import numpy as np
import re
import pickle
import matplotlib.pyplot as  plt
%matplotlib inline
warnings.filterwarnings("ignore", category=DeprecationWarning) # 경고 알림이 뜨면 모두 무시

### 2. 텍스트 전처리 함수 만들기 
: 네이버 기사를 pickle을 사용해서 저장했으니, 불러와야하는데, 
데이터를 저번에 페이지 단위로 가져온 네이버 기사를 불러와서
1차원 리스트로 바꿔줘야 한다. .

In [23]:
def read_documents(input_file_name):
    """문서들을 주어진 이름의 파일로부터 읽어들여 돌려준다."""
    
    corpus = []
    
    with open(input_file_name, 'rb') as f: #rb : pickle파일을 읽기 위해서 read as binary
        temp_corpus = pickle.load(f) #일단 불러와야 한다. 
        
    for page in temp_corpus:
        corpus += page #page를 하나의 corpus에 다 붙여줌. 
    
    return corpus

def text_cleaning(docs):
    # 한국어를 제외한 글자를 제거하는 함수.
    cleaned_docs = []
    
    for doc in docs:
        temp_doc = re.sub("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", "", doc)
        cleaned_docs.append(temp_doc)

    return cleaned_docs

In [24]:
def define_stopwords(path): 
    #특수문자 추가 
    SW = set()
    # 불용어를 추가하는 방법 1.
    for i in string.punctuation: #string.puctuation : 특수 문자
        SW.add(i)

    # 불용어를 추가하는 방법 2.
    # stopwords-ko.txt에 직접 추가
    
    with open(path) as f:
        for word in f:
            SW.add(word)

    return SW

** 기사의 특성상 텍스트가 지저분 할 수 있다. 
그럴 때, 명사 추출방식, 즉 (키워드 추출한다는 느낌으로) 명사만 뽑아버리고 나머지를 다 버리면, 자연스럽게 특수문자가 제거된다. 
이렇듯 보통 지저분한 텍스트인 경우, 명사 추출방식을 많이 쓴다. 

In [25]:
def text_tokenizing(corpus, tokenizer):
    #명사 추출/ 형태소 분석 두 가지를 선택할 수 있게 만들어주는 햠수를 만들어보자.
    #tqdm을 이용하여 진행과정을 볼 수 있게 만들어본다. 
   
    mecab = Mecab() 
    token_corpus = []
    
     #tokenizer : 뭘 기준으로 자를 건지를 명시해 주는 변수
    if tokenizer == "noun":
        for n in tqdm_notebook(range(len(corpus)), desc="Noun Preprocessing"): #desc= : progressing bar 위에 이름테그
            token_text = mecab.nouns(corpus[n]) #corpus[n]에서 명사 추출하고, 
            token_text = [word for word in token_text if word not in SW] 
            #filtering : mecab.nouns를 통해 명사를 추출(word)했으면, 그 word가 추출한 것에 있을 때 
                
            token_corpus.append(token_text) #다 했으면, append
            
    elif tokenized == "morph": #형태소 단위
        for n in tqdm_notebook(range(len(corpus)), desc="Morph Preprocessing"):
            token_text = mecab.morphs(corpus[n])
            token_text = [word for word in token_text if word not in SW]
            token_corpus.append(token_text)

    elif tokenizer == "word": #단어 단위
        for n in tqdm_notebook(range(len(corpus)), desc="Word Preprocessing"):
            token_text = corpus[n].split()
            token_text = [word for word in token_text if word not in SW]
            token_corpus.append(token_text)
        

    return token_corpus

#함수를 불러오는 코드
input_file_name = "/Users/jeonghyeonjeong/for github/머신러닝_데이터분석A-Z_패스트캠퍼스/텍스트마이닝을 활용한 카카오톡 감성분석/naver_news_content.pk"
documents = read_documents(input_file_name)
SW = define_stopwords("/Users/jeonghyeonjeong/for github/머신러닝_데이터분석A-Z_패스트캠퍼스/텍스트마이닝을 활용한 카카오톡 감성분석/stopwords-ko.txt") #불용어
cleaned_text = text_cleaning(documents)
tokenized_text = text_tokenizing(cleaned_text, tokenizer="noun") #tokenizer= "noun" or "word" or "morph" 

In [27]:
#결과 확인
print(tokenized_text)

[['본문', '내용', '플레이어', '플레이어', '오류', '우회', '함수', '추가', '발전', '자료', '과학', '기술', '정보', '통신부', '유럽', '일본', '미국', '등', '주요', '국가', '수년', '전', '인공지능', '관련', '저작', '논의', '시작', '이', '국가', '발전', '빅', '데이터', '등', '관련', '저작', '문제', '해결', '인식', '빅', '데이터', '소재', '머신', '러닝', '딥', '러닝', '방식', '학습', '특정', '결과', '구현', '기술', '이', '데이터', '수집', '저장', '처리', '과정', '데이터', '복제', '전송', '등', '과정', '필요', '빅', '데이터', '분석', '통계', '규칙', '경향', '등', '가치', '정보', '것', '텍스트', '데이터', '마이닝', '분석', '대상', '데이터', '타인', '저작물', '포함', '저작', '침해', '문제', '발생', '수', '빅', '데이터', '활용', '제한', '요소', '저작물', '활용', '저작', '동의', '빅', '데이터', '포함', '저작', '동의', '것', '사실', '가능', '시간', '비용', '소모', '때문', '영국', '프랑스', '독일', '등', '유럽', '주요', '국가', '문제', '인식', '년', '영국', '시작', '저작권법', '등', '관련', '법', '특별', '규정', '도입', '나라', '내용', '차이', '공통', '것', '상업', '연구', '목적', '경우', '저작', '동의', '저작물', '활용', '수', '점', '유럽연합', '올해', '월', '디지털', '단일', '시장', '저작', '유럽', '의회', '위원회', '지침', '을', '입법', '지침', '마련', '타인', '저작물', '활용', '은', '학문', '연구', '목적', '문화유산', '기구

### 3. 토픽 모델링에 사용할 함수들 확인하기 - 중요
LDA의 input으로 들어가는 것
    1. dictionary
    2. corpus

In [29]:
#LDA 도 document term matrix이라 BoW바꿔줘야함 
#dictionary : key)숫자, value)각 단어 => 이 단어가 어디에 있는지 효율적으로 알기 위해 이런 형식으로 만들어주면 좋다. 
dictionary = corpora.Dictionary(tokenized_text)

#dictionary 만들었으면 이제 document-term matrix을 생성
corpus = [dictionary.doc2bow(text) for text in tokenized_text]

###이 부분 잘 모르겠어서.. 공부 필요##

In [30]:
#Dictionary 확인
print(dictionary)

Dictionary(1096 unique tokens: ['가능', '가치', '감정', '개념', '개정']...)


In [37]:
#corpus 확인
corpus[0][:5]  #BoW안에 원소가 (index, 개수) 일단 너무 쭉 나오니까 5개만 출력

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 4)]

In [40]:
#BoW 말고, TFIDFmodel의 document-term matrix 생성(하고싶다면) #TFIDF..?
tfidf = models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]
corpus_tfidf[0][:5] #(index, tfidf값)

[(0, 0.012056637504869792),
 (1, 0.024113275009739584),
 (2, 0.036169912514609374),
 (3, 0.024113275009739584),
 (4, 0.1446796500584375)]

In [42]:
#근데 여기선 BoW모델 사용할 것임
#LDA model 만들기
model = models.ldamodel.LdaModel(corpus, num_topics = 3, id2word = dictionary) #models.ldamodle.LdaModel 암기하기
#corpus(document-term model), num_topic : 토픽 몇개로 할 건지, id2word : 단어가 어디있는지 dictionary word로 mapping되어 있는 것으로 제공

In [43]:
#LDA model 결과 확인
model.show_topic(1, 10)#model.show_topic(topic_no, num_word)  : 즉, (몇 번째 topic의, top num_word개를 보고싶어)

[('기업', 0.011318243),
 ('년', 0.011303322),
 ('업체', 0.010951964),
 ('수', 0.009682038),
 ('등', 0.009122378),
 ('데이터', 0.009040101),
 ('경제', 0.008251301),
 ('것', 0.0073610353),
 ('공급', 0.0071919444),
 ('정보', 0.0066026947)]

### 4. 토픽 모델링을 추가하여 코드 완성하기

In [44]:
# 토픽 개수, 키워드 개수를 정해주는 변수를 추가.
NUM_TOPICS = 3

NUM_TOPIC_WORDS = 30


def build_doc_term_mat(documents):
    # 문서-단어 행렬 만들어주는 함수.
    print("Building document-term matrix.")
    dictionary = corpora.Dictionary(documents)
    corpus = [dictionary.doc2bow(document) for document in documents]
        
    return corpus, dictionary


def print_topic_words(model):

    # 토픽 모델링 결과를 출력해 주는 함수.
    print("\nPrinting topic words.\n")
    
    for topic_id in range(model.num_topics):
        topic_word_probs = model.show_topic(topic_id, NUM_TOPIC_WORDS)
        print("Topic ID: {}".format(topic_id))
        
        for topic_word, prob in topic_word_probs:
            print("\t{}\t{}".format(topic_word, prob))
            
        print("\n")

# document-term matrix를 만들고,
corpus, dictionary = build_doc_term_mat(tokenized_text)
# LDA를 실행.
model = models.ldamodel.LdaModel(corpus, num_topics=NUM_TOPICS, id2word=dictionary, alpha="auto", eta="auto")
# 결과를 출력.
print_topic_words(model)

Building document-term matrix.

Printing topic words.

Topic ID: 0
	업체	0.02248646877706051
	기업	0.015983860939741135
	공급	0.014142648316919804
	데이터	0.01234776247292757
	정보	0.011531614698469639
	수	0.010496069677174091
	캐스팅	0.010406330227851868
	폼	0.007227912079542875
	것	0.007168847601860762
	플랫	0.007075203116983175
	서비스	0.0068495068699121475
	년	0.00677415169775486
	가격	0.0066842990927398205
	일	0.006595319136977196
	엔	0.00648255692794919
	때문	0.005646990146487951
	수요	0.005389566998928785
	빅	0.0052745831198990345
	신뢰	0.0051464419811964035
	등	0.005105292424559593
	내용	0.0049115498550236225
	플레이어	0.004879787098616362
	대표	0.004859576001763344
	경제	0.004380758386105299
	본문	0.00416865898296237
	한국	0.004096561577171087
	분석	0.004055002238601446
	제공	0.0038333425763994455
	마이닝	0.0037648871075361967
	이용	0.0037244486156851053


Topic ID: 1
	년	0.01163471583276987
	데이터	0.007955155335366726
	등	0.007590803783386946
	내용	0.007115837652236223
	경제	0.006759092211723328
	빅	0.006471374537795782
	산업	0.006038875319063

### 5. pyLDAvis을 통한 토픽 모델링 결과 시각화하기

In [48]:
!pip install pyLDAvis #pyLDAvis설치 LDAvis를 python으로 포팅

In [49]:
# pyLDAvis 불러오기 #암기
import pyLDAvis
import pyLDAvis.gensim

# pyLDAvis를 jupyter notebook에서 실행할 수 있게 활성화. #jupyter에서 사용할 때 암기
pyLDAvis.enable_notebook()

# pyLDAvis 실행. #암기
data = pyLDAvis.gensim.prepare(model, corpus, dictionary)
data #print치면 안되고 그냥 data로 실행

/Users/jeonghyeonjeong/anaconda3/envs/test/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.027717 -0.007749       1        1  47.164268
1     -0.019122  0.008759       2        1  31.643513
0      0.046839 -0.001010       3        1  21.192219, topic_info=     Category       Freq Term      Total  loglift  logprob
708   Default  18.000000   업체  18.000000  30.0000  30.0000
952   Default  11.000000   공급  11.000000  29.0000  29.0000
245   Default  21.000000   기업  21.000000  28.0000  28.0000
1023  Default   9.000000  캐스팅   9.000000  27.0000  27.0000
140   Default  13.000000   정보  13.000000  26.0000  26.0000
...       ...        ...  ...        ...      ...      ...
74     Topic3   2.672563   분석  11.753654   0.0704  -5.5078
643    Topic3   2.699954   한국  13.552250  -0.0618  -5.4976
205    Topic3   2.887261   경제  27.736687  -0.7109  -5.4305
48     Topic3   2.481354  마이닝   8.859549   0.2788  -5.5820
113    Topic3   2.454702   이용   9.965832   0.1504  -5.5928

[204 rows x 6 columns], token_table=      Topic      Freq Term
term                      
944       3  0.573033    ㄱ
187       1  0.301067   가격
187       3  0.602134   가격
0         1  0.444250   가능
0         2  0.444250   가능
...     ...       ...  ...
176       2  0.195891   해결
176       3  0.391783   해결
1088      1  0.285328  화장품
1088      2  0.427992  화장품
1088      3  0.285328  화장품

[356 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 2, 1])